In [85]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [86]:
data = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
y, x= data.iloc[:, 0], data.iloc[:, 1:]
test = torch.from_numpy(np.array(test).astype(np.float32))
print(x.shape, y.shape)
x, y = np.array(x), np.array(y)
x, x_vl, y, y_vl = train_test_split(x, y, test_size = 0.3, shuffle = True)
print(x.shape, x_vl.shape)
x, y, x_vl, y_vl = torch.from_numpy(x.astype(np.float32)), torch.from_numpy(y.astype(np.float32)), torch.from_numpy(x_vl.astype(np.float32)), torch.from_numpy(y_vl.astype(np.float32))
x = x.view(-1, 1, 28, 28)
x_vl = x_vl.view(-1, 1, 28, 28)
test = test.view(-1, 1, 28, 28)
print(x.shape, x_vl.shape, test.shape, y.shape)
#.imshow(x[26].reshape(28, 28))

In [87]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv = nn.Sequential( 
            # size: 28*28
            nn.Conv2d(1,8,3,1,1), # in_channels out_channels kernel_size stride padding
            nn.ReLU(),
            nn.Conv2d(8,16,3,1,1), 
            nn.ReLU(),
            nn.MaxPool2d(2),
            # size: 14*14
            nn.Conv2d(16,16,3,1,1), 
            nn.ReLU(),
            nn.Conv2d(16,8,3,1,1), 
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Sequential(
            # size: 7*7
            nn.Linear(8*7*7,256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256,256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256,10)
        )

    
    def forward(self, img):
        x = self.conv(img)
        o = self.fc(x.view(x.shape[0],-1))
        return o


In [88]:
bs = 64
trainDS = TensorDataset(x, y)
testDS = TensorDataset(test)
validDS = TensorDataset(x_vl, y_vl)

trainDL = DataLoader(trainDS, batch_size = bs, shuffle = True)
testDL = DataLoader(testDS, batch_size = bs, shuffle = True)
validDL = DataLoader(validDS, batch_size = bs, shuffle = True)

In [91]:
criterion = nn.CrossEntropyLoss()
net = Net()
optim = torch.optim.SGD(net.parameters(), lr = 0.01)
epochs = 22

In [92]:
for epoch in range(epochs):
    epoc_train_loss = 0.0
    epoc_train_corr = 0.0
    epoc_valid_corr = 0.0
    print('Epoch:{}/{}'.format(epoch,epochs))
    
    for data in trainDL:
        
        images,labels = data
        outputs = net(images)               
        optim.zero_grad()
        loss = criterion(outputs,labels.long())
        loss.backward()
        optim.step()
        
        epoc_train_loss += loss.data
        outputs = torch.max(outputs.data,1)[1]
        epoc_train_corr += torch.sum(outputs==labels.data)
    
    with torch.no_grad():
        for data in validDL:

            images,labels = data
            labels = labels


            outputs = net(images)
            outputs = torch.max(outputs.data,1)[1]

            epoc_valid_corr += torch.sum(outputs==labels.data)
    
    
    print("loss is :{:.4f},Train Accuracy is:{:.4f}%,Test Accuracy is:{:.4f}%".format(epoc_train_loss/len(trainDS),100*epoc_train_corr/len(trainDS),100*epoc_valid_corr/len(validDS)))

In [93]:
y_pred = net(test)
y_pred = torch.max(y_pred, 1)[1]
print(y_pred.shape)

In [102]:
out = np.array(y_pred)
out = {'ImageId': range(1, len(out) + 1), 'Label': out}
out = pd.DataFrame(out)
print(out.shape)
out.to_csv('submission.csv', index = False)